In [185]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import LabelEncoder
import pandas as pd



In [186]:
df = pd.read_csv('social_media_dataset.csv')

In [187]:
df.head()

,post_id,timestamp,day_of_week,platform,user_id,location,language,text_content,hashtags,mentions,...,comments_count,impressions,engagement_rate,brand_name,product_name,campaign_name,campaign_phase,user_past_sentiment_avg,user_engagement_growth,buzz_change_rate
0,kcqbs6hxybia,2024-12-09 11:26:15,Monday,Instagram,user_52nwb0a6,"Melbourne, Australia",pt,Just tried the Chromebook from Google. Best pu...,#Food,NaN,...,701,18991,0.19319,Google,Chromebook,BlackFriday,Launch,0.0953,-0.3672,19.1
1,vkmervg4ioos,2024-07-28 19:59:26,Sunday,Twitter,user_ucryct98,"Tokyo, Japan",ru,Just saw an ad for Microsoft Surface Laptop du...,"#MustHave, #Food","@CustomerService, @BrandCEO",...,359,52764,0.05086,Microsoft,Surface Laptop,PowerRelease,Post-Launch,0.1369,-0.4510,-42.6
2,memhx4o1x6yu,2024-11-23 14:00:12,Saturday,Reddit,user_7rrev126,"Beijing, China",ru,What's your opinion about Nike's Epic React? ...,"#Promo, #Food, #Trending",NaN,...,643,8887,0.45425,Nike,Epic React,BlackFriday,Post-Launch,0.2855,-0.4112,17.4
3,bhyo6piijqt9,2024-09-16 04:35:25,Monday,YouTube,user_4mxuq0ax,"Lagos, Nigeria",en,Bummed out with my new Diet Pepsi from Pepsi! ...,"#Reviews, #Sustainable","@StyleGuide, @BrandSupport",...,743,6696,0.42293,Pepsi,Diet Pepsi,LaunchWave,Launch,-0.2094,-0.0167,-5.5
4,c9dkiomowakt,2024-09-05 21:03:01,Thursday,Twitter,user_l1vpox2k,"Berlin, Germany",hi,Just tried the Corolla from Toyota. Absolutely...,"#Health, #Travel","@BrandSupport, @InfluencerName",...,703,47315,0.08773,Toyota,Corolla,LocalTouchpoints,Launch,0.6867,0.0807,38.8


In [188]:
df.describe()

,sentiment_score,toxicity_score,likes_count,shares_count,comments_count,impressions,engagement_rate,user_past_sentiment_avg,user_engagement_growth,buzz_change_rate
count,12000.000000,12000.000000,12000.00000,12000.000000,12000.00000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,0.000553,0.503868,2490.72025,1007.167167,504.34575,49811.338500,0.278137,0.001472,0.000998,0.729692
std,0.583563,0.288198,1441.53253,575.072282,288.68416,28930.289451,1.149206,0.576627,0.289940,57.787219
min,-0.999800,0.000000,0.00000,0.000000,0.00000,130.000000,0.001880,-0.999600,-0.499900,-99.900000
25%,-0.503200,0.251400,1236.00000,510.000000,253.00000,24716.500000,0.049100,-0.495975,-0.248400,-48.700000
50%,-0.006200,0.505950,2496.00000,1018.000000,503.00000,49674.000000,0.080605,0.001950,0.002800,0.900000
75%,0.513525,0.756200,3723.25000,1501.000000,755.00000,74815.000000,0.163123,0.501725,0.250700,50.100000
max,0.999900,0.999900,5000.00000,2000.000000,1000.00000,99997.000000,32.211710,0.999400,0.499900,99.900000


In [189]:
df.shape

(12000, 28)

In [190]:
df = df.dropna()
df = df.drop('user_id', axis=1)


In [191]:
rename_dict = {
    'timestamp': 'data_e_hora',
    'day_of_week': 'dia_da_semana',
    'platform': 'plataforma',
    'location': 'localizacao',
    'language': 'idioma',
    'text_content': 'conteudo_de_texto',
    'hashtags': 'hastags',
    'mentions': 'mencoes',
    'comments_count': 'comentarios',
    'likes_count': 'contagem',
    'impressions': 'impressoes',
    'engagement_rate': 'taxa_de_engajamento',
    'brand_name': 'marca_registrada',
    'product_name': 'nome_do_produto',
    'campaign_name': 'nome_da_campanha',
    'campaign_phase': 'fase_da_campnha',
    'user_past_sentiment_avg': 'sentimento_medio_do_usuario_anterior',
    'user_engagement_growth': 'crescimento_do_engajamento_do_usuario',
    'buzz_change_rate': 'taxa_de_mudanca_de_buzz'
}



In [192]:
colunas_object = df.select_dtypes(include=['object']).columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8059 entries, 1 to 11999
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   post_id                  8059 non-null   object 
 1   timestamp                8059 non-null   object 
 2   day_of_week              8059 non-null   object 
 3   platform                 8059 non-null   object 
 4   location                 8059 non-null   object 
 5   language                 8059 non-null   object 
 6   text_content             8059 non-null   object 
 7   hashtags                 8059 non-null   object 
 8   mentions                 8059 non-null   object 
 9   keywords                 8059 non-null   object 
 10  topic_category           8059 non-null   object 
 11  sentiment_score          8059 non-null   float64
 12  sentiment_label          8059 non-null   object 
 13  emotion_type             8059 non-null   object 
 14  toxicity_score           805

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8059 entries, 1 to 11999
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   post_id                  8059 non-null   object 
 1   timestamp                8059 non-null   object 
 2   day_of_week              8059 non-null   object 
 3   platform                 8059 non-null   object 
 4   location                 8059 non-null   object 
 5   language                 8059 non-null   object 
 6   text_content             8059 non-null   object 
 7   hashtags                 8059 non-null   object 
 8   mentions                 8059 non-null   object 
 9   keywords                 8059 non-null   object 
 10  topic_category           8059 non-null   object 
 11  sentiment_score          8059 non-null   float64
 12  sentiment_label          8059 non-null   object 
 13  emotion_type             8059 non-null   object 
 14  toxicity_score           805

In [194]:
le = LabelEncoder()
for coluna in colunas_object:
    df[coluna] = le.fit_transform(df[coluna])



In [195]:
X = df.drop(['platform'], axis=1)
y = df['platform']



In [196]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [198]:
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)



SVC(kernel='linear')

In [199]:
y_pred = svm_model.predict(X_test)


In [200]:
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia:", accuracy)
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))



Acurácia: 0.20843672456575682
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.22      0.20      0.21       334
           1       0.23      0.25      0.24       313
           2       0.20      0.15      0.17       319
           3       0.16      0.19      0.17       304
           4       0.23      0.25      0.24       342

    accuracy                           0.21      1612
   macro avg       0.21      0.21      0.21      1612
weighted avg       0.21      0.21      0.21      1612



In [201]:
# Crie um dicionário com todos os nomes de colunas usados em X (exceto 'platform')
colunas_modelo = [col for col in df.columns if col != 'platform']

# Preencha os valores do novo usuário (ajuste conforme necessário)
valores_novo_usuario = {
	'post_id': 0,
	'timestamp': 1,
	'day_of_week': 2,
	'location': 4,
	'language': 5,
	'text_content': 6,
	'hashtags': 7,
	'mentions': 8,
	'keywords': 0,
	'topic_category': 0,
	'sentiment_score': 0.0,
	'sentiment_label': 0,
	'emotion_type': 0,
	'toxicity_score': 0.0,
	'likes_count': 10,
	'shares_count': 0,
	'comments_count': 9,
	'impressions': 11,
	'engagement_rate': 12,
	'brand_name': 13,
	'product_name': 14,
	'campaign_name': 15,
	'campaign_phase': 16,
	'user_past_sentiment_avg': 17,
	'user_engagement_growth': 18,
	'buzz_change_rate': 19
}

# Garante que todas as colunas estejam presentes e na ordem correta
novo_usuario = pd.DataFrame([{col: valores_novo_usuario.get(col, 0) for col in colunas_modelo}])

# Padronize os dados
novo_usuario_scaled = scaler.transform(novo_usuario)

# Faça a predição
predicao = svm_model.predict(novo_usuario_scaled)
print("Rede social prevista:", predicao)


Rede social prevista: [4]
